In [16]:
from optibook.synchronous_client import Exchange
import time
import logging

In [17]:
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


In [18]:
exchange = Exchange()
_ = exchange.connect()

2023-03-05 05:30:55,769 [asyncio   ] [MainThread  ] Using selector: EpollSelector


In [4]:
def getgreen():
    wind  = exchange.get_last_price_book('C2_WIND_LTD')
    solar = exchange.get_last_price_book('C2_SOLAR_CO')
    etf   = exchange.get_last_price_book('C2_GREEN_ENERGY_ETF')
   
    # returns tuple containing ask price and vol
    try:
        windask = (wind.asks[0].price, wind.asks[0].volume)
        solarask = (solar.asks[0].price, solar.asks[0].volume)
        etfask = (etf.asks[0].price, etf.asks[0].volume)

        # returns tuple containing bid price and vol
        windbid = (wind.bids[0].price, wind.bids[0].volume)
        solarbid = (solar.bids[0].price, solar.bids[0].volume)
        etfbid = (etf.bids[0].price, etf.bids[0].volume)
        
    except IndexError:
        print('Skipping cycle - Value fetch error')
        return False, False
    return (windask, solarask, etfask),(windbid,solarbid,etfbid) 


In [5]:
def getfossil():
    oil  = exchange.get_last_price_book('C1_OIL_CORP')
    gas = exchange.get_last_price_book('C1_GAS_INC')
    etf   = exchange.get_last_price_book('C1_FOSSIL_FUEL_ETF')
    
    # returns tuple containing ask price and vol
    try:
        oilask = (oil.asks[0].price, oil.asks[0].volume)
        gasask = (gas.asks[0].price, gas.asks[0].volume)
        etfask = (etf.asks[0].price, etf.asks[0].volume)

        # returns tuple containing bid price and vol
        oilbid = (oil.bids[0].price, oil.bids[0].volume)
        gasbid = (gas.bids[0].price, gas.bids[0].volume)
        etfbid = (etf.bids[0].price, etf.bids[0].volume)
        
    except IndexError:
        print('Skipping cycle - Value fetch error')
        return False, False

    return (oilask, gasask, etfask),(oilbid,gasbid,etfbid) 
   

In [6]:
        
def order_wind_bid(price,volume,type1): # give price , volume , type as 'limit' or 'ioc'
    exchange.insert_order('C2_WIND_LTD', price=price, volume=volume, side='bid', order_type=type1)
def order_wind_ask(price,volume,type1):
    exchange.insert_order('C2_WIND_LTD', price=price, volume=volume, side='ask', order_type=type1)
    
    
def order_solar_bid(price,volume,type1): # give price , volume , type as 'limit' or 'ioc'
    exchange.insert_order('C2_SOLAR_CO', price=price, volume=volume, side='bid', order_type=type1)
def order_solar_ask(price,volume,type1):
    exchange.insert_order('C2_SOLAR_CO', price=price, volume=volume, side='ask', order_type=type1)
    
    
def order_GreenETF_bid(price,volume,type1): # give price , volume , type as 'limit' or 'ioc'
    exchange.insert_order('C2_GREEN_ENERGY_ETF', price=price, volume=volume, side='bid', order_type=type1)
def order_GreenETF_ask(price,volume,type1):
    exchange.insert_order('C2_GREEN_ENERGY_ETF', price=price, volume=volume, side='ask', order_type=type1)
    
    
            
def order_oil_bid(price,volume,type1): # give price , volume , type as 'limit' or 'ioc'
    exchange.insert_order('C1_OIL_CORP', price=price, volume=volume, side='bid', order_type=type1)
def order_oil_ask(price,volume,type1):
    exchange.insert_order('C1_OIL_CORP', price=price, volume=volume, side='ask', order_type=type1)
    
    
def order_gas_bid(price,volume,type1): # give price , volume , type as 'limit' or 'ioc'
    exchange.insert_order('C1_GAS_INC', price=price, volume=volume, side='bid', order_type=type1)
def order_gas_ask(price,volume,type1):
    exchange.insert_order('C1_GAS_INC', price=price, volume=volume, side='ask', order_type=type1)
    
    
def order_FossilETF_bid(price,volume,type1): # give price , volume , type as 'limit' or 'ioc'
    exchange.insert_order('C1_FOSSIL_FUEL_ETF', price=price, volume=volume, side='bid', order_type=type1)
def order_FossilETF_ask(price,volume,type1):
    exchange.insert_order('C1_FOSSIL_FUEL_ETF', price=price, volume=volume, side='ask', order_type=type1)

In [7]:
def liquify (asks,bids):
    v=30
    time.sleep(0.1)
    time1=None
    v_bids=min(v,bids[1][1],bids[0][1])
    v_asks=min(v,asks[1][1],asks[0][1])
    if (len(exchange.get_outstanding_orders("C2_GREEN_ENERGY_ETF"))==0) and abs(exchange.get_positions()["C2_GREEN_ENERGY_ETF"]) <450 :
        if (asks[2][0] - bids[2][0]) > 0.2: 
            gask = exchange.insert_order('C2_GREEN_ENERGY_ETF',price= asks[2][0]-0.1,volume=2*v_bids, side='ask', order_type='limit')
            gbid = exchange.insert_order('C2_GREEN_ENERGY_ETF',price= asks[2][0]-0.1,volume=2*v_bids, side='bid', order_type='limit')
            time1.time()
            
        else:
            gask = exchange.insert_order('C2_GREEN_ENERGY_ETF',price= asks[2][0],volume=2*v_bids, side='ask', order_type='limit')
            gbid = exchange.insert_order('C2_GREEN_ENERGY_ETF',price= asks[2][0],volume=2*v_bids, side='bid', order_type='limit')
            time1.time()
    else :
        print("not liquified")
            
    time.sleep(0.1)
    if gask.order_id not in exchange.get_outstanding_orders("C2_GREEN_ENERGY_ETF"):
        order_solar_bid(bids[1][0],v_bids,'ioc')
        order_wind_bid (bids[0][0],v_bids,'ioc')
    if gbid.order_id not in exchange.get_outstanding_orders("C2_GREEN_ENERGY_ETF"):
        order_solar_bid(asks[1][0],v_asks,'ioc')
        order_wind_bid (asks[0][0],v_asks,'ioc')
    return gask,gbid,time1 
    

                    
        
        
        

In [8]:
def flipper_calc(asks,bids):# returns the flip value of the direction of the market #
    
    if ((asks[0][0]+asks[1][0])/2 < bids[2][0]): # asks 
        return 1
    else : 
        return -1

In [9]:
def trademargingreen(asks, bids, typ):
    global flip
    v = min(globalv, min(2*(min(asks[0][1], asks[1][1])), bids[2][1]))
    if (asks[0][0]+asks[1][0])/2 < bids[2][0]:
        flip[0] = 1
        print('Trade: Buying solar and wind, selling etf')
        # sell 2n etfs
        order_GreenETF_ask(bids[2][0]-0.1, 2*v, typ)


        # buy n solar and n wind
        order_wind_bid(asks[0][0], v, typ)
        order_solar_bid(asks[1][0], v, typ)

    elif (bids[0][0]+bids[1][0])/2 > asks[2][0]:
        flip[0] = -1
        print('Trade: Selling solar and wind, buying etf')
        # sell n solar and n wind
        order_wind_ask(bids[0][0], v, typ)
        order_solar_ask(bids[1][0], v, typ)

        # buy 2n etfs
        order_GreenETF_bid(asks[2][0]+0.1, 2*v, typ)
        

In [10]:
def trademarginfossil(asks, bids, typ):
    global flip 
    v = min(globalv, min(2*min(asks[0][1], asks[1][1]), bids[2][1]))
    
    if (asks[0][0]+asks[1][0])/2 < bids[2][0]:
        flip[1] = 1
        print('Trade: Buying oil and gas, selling etf')
        # sell 2n etfs
        order_FossilETF_ask(bids[2][0], 2*v, typ)


        # buy n solar and n wind
        order_oil_bid(asks[0][0], v, typ)
        order_gas_bid(asks[1][0], v, typ)

    elif (bids[0][0]+bids[1][0])/2 > asks[2][0]:
        flip[1] = -1
        print('Trade: Selling oil and gas, buying etf')
        # sell n solar and n wind
        order_oil_ask(bids[0][0], v, typ)
        order_gas_ask(bids[1][0], v, typ)

        # buy 2n etfs
        order_FossilETF_bid(asks[2][0], 2*v, typ)

In [11]:
def exit_market():
    MIN_SELLING_PRICE = 0.10
    MAX_BUYING_PRICE = 100000.00

    positions = exchange.get_positions()
    pnl = exchange.get_pnl()

    print(f'Positions before: {positions}')
    print(f'\nPnL before: {pnl:.2f}')

    print(f'\nTrading out of positions')
    for iid, pos in positions.items():
        if pos > 0:
            print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
            exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
        elif pos < 0:
            print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
            exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
        else:
            print(f'-- No initial position in {iid}, skipping..')

    positions = exchange.get_positions()
    pnl = exchange.get_pnl()
    print(f'\nPositions after: {positions}')
    print(f'\nPnL after: {pnl:.2f}')

    

In [12]:
def partial_exit_market():
    MIN_SELLING_PRICE = 0.10
    MAX_BUYING_PRICE = 100000.00

    positions = exchange.get_positions() # dict of id , position 
    pnl = exchange.get_pnl()

    print(f'Positions before: {positions}')
    print(f'\nPnL before: {pnl:.2f}')

    print(f'\nTrading out of positions')
    for iid, pos in positions.items():
        
        if pos > 0:
            print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
            MIN_SELLING_PRICE=exchange.get_last_price_book(iid).bids[0].price
            exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='limit')
        elif pos < 0:
            MAX_BUYING_PRICE=exchange.get_last_price_book(iid).asks[0].price
            print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
            exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='limit')
        else:
            print(f'-- No initial position in {iid}, skipping..')


    positions = exchange.get_positions()
    pnl = exchange.get_pnl()
    print(f'\nPositions after: {positions}')
    print(f'\nPnL after: {pnl:.2f}')

In [13]:
def partial_half_exit_market():
    MIN_SELLING_PRICE = 0.10
    MAX_BUYING_PRICE = 100000.00

    positions = exchange.get_positions() # dict of id , position 
    pnl = exchange.get_pnl()

    print(f'Positions before: {positions}')
    print(f'\nPnL before: {pnl:.2f}')

    print(f'\nTrading out of positions')
    for iid, pos in positions.items():
        pos=pos//2
        if pos > 0:
            print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
            MIN_SELLING_PRICE=exchange.get_last_price_book(iid).bids[0].price
            exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='limit')
        elif pos < 0:
            MAX_BUYING_PRICE=exchange.get_last_price_book(iid).asks[0].price
            print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
            exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='limit')
        else:
            print(f'-- No initial position in {iid}, skipping..')

    positions = exchange.get_positions()
    pnl = exchange.get_pnl()
    print(f'\nPositions after: {positions}')
    print(f'\nPnL after: {pnl:.2f}')

In [15]:
##  MAIN 

#global flag 
flag = False
flip = [0,0]
totalexp = 0
globalv = 20 # volume to trade at
#for i in range(100):
while(True):
    
    # GET BEST ASKS AND BIDS FOR ALL 6 INSTRUMENTS
    asks1, bids1 = getgreen()    
    asks2, bids2 = getfossil()
    
    #####################################################
     
    if flag: ## continue position only WHEN market is liquid and flipped 
        if asks1 != False and asks2 !=False:
             ## GREEN
            f = flipper_calc(asks1, bids1)
            if f != flip[0]:
                time.sleep(0.75)
                f = flipper_calc(asks1, bids1)
                if f != flip[0]:
                    flag = False
                
             ## FOSSIL
            f = flipper_calc(asks2, bids2)
            if f != flip[1]:
                time.sleep(0.75)
                f = flipper_calc(asks2, bids2)
                if f != flip[1]:
                    flag = False
            continue

    
    # TRADE IF VALID DATA
    
    if (asks1 and asks2) is not False:
        trademargingreen(asks1,bids1, 'ioc')  
        trademarginfossil(asks2,bids2, 'ioc')   
        
        
#     ######### LIQUIFYING GREEN ########
#     if asks1 is not False:
#         #green_spread = (asks1[2][0]-bids1[2][0])/((asks1[2][0]+bids1[2][0])/2)
#         green_spread = (asks1[2][0]-bids1[2][0])*100/(asks1[2][0])
#     else:
#         green_spread = 0
    
#     asks1, bids1 = getgreen()    
    
#     if (asks1 is not False) and (bids1 is not False):
#         gask,gbid,time1 = liquify(asks1,bids1)
    
    
    
    
        
    #####################################################
    # code for positions to set flag to true
    
    pos = exchange.get_positions()
    #print(pos)
    for i in pos:
        if abs(pos[i]) >300:
            print("Pause Initiated")
            flag=True
    
    #####################################################
   
    
      

#     ##### DELETING LIMIT ORDERS ##########
    
#     if time1 is not None:
#         if (time1-time.time()>1):
#             for i in exchange.get_positions(): # deleting the limit orders 
#                 exchange.delete_orders(i) # resets exchange every 2 secs 
    
    time.sleep(0.1)


Trade: Selling solar and wind, buying etf
Trade: Selling solar and wind, buying etf
Trade: Selling solar and wind, buying etf
Trade: Selling solar and wind, buying etf
Trade: Selling solar and wind, buying etf
Trade: Selling solar and wind, buying etf
Trade: Selling solar and wind, buying etf
Trade: Selling solar and wind, buying etf
Pause Initiated
Trade: Selling solar and wind, buying etf
Pause Initiated
Pause Initiated
Trade: Selling solar and wind, buying etf
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Skipping cycle - Value fetch error
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Trade: Selling solar and wind, buying etf
Pause Initiated
Trade: Selling solar and wind, buying etf
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Pause Initiated
Trade: Sellin

2023-03-05 05:27:18,489 [client    ] [Thread-4    ] Forcing a disconnect due to an error: Pre-trade position limit (-500) breached on instrument C2_GREEN_ENERGY_ETF. Old position=-475 New worst-case position=-515.


Trade: Buying solar and wind, selling etf


KjException: (remote):0: failed: remote exception: std::exception: Pre-trade position limit (-500) breached on instrument C2_GREEN_ENERGY_ETF. Old position=-475 New worst-case position=-515
stack: 7f5f67be6cc0 7f5f67be5540 7f5f67b696d6 7f5f67b57612 7f5f67b57749 7f5f67b4cc74 7f5f67ab5639 7f5f67b2e673 7f5f91f3fcc0 7f5f91eafe4e 7f5f84a14cba 7f5f84a16293 7f5f91e9f0a2 7f5f91f3be68 7f5f91f45569 7f5f91f45992 7f5f91f3a2c6 7f5f91f45569 7f5f91f45992 7f5f91f3a2c6 7f5f91f45569 7f5f91f45992 7f5f91f3a2c6 7f5f91f45569 7f5f91f45992 7f5f91f3a2c6 7f5f91f46b01 7f5f91e9f31d 7f5f91e9f421 7f5f91e9f0a2 7f5f91f3be68 7f5f91f45569

In [19]:
exchange.get_positions()

{'C1_GAS_INC': 20,
 'C1_OIL_CORP': 20,
 'C1_FOSSIL_FUEL_ETF': -110,
 'C2_GREEN_ENERGY_ETF': -475,
 'C2_SOLAR_CO': 195,
 'C2_WIND_LTD': 235}

In [20]:
exchange.get_pnl()

1817.6383999999962

In [21]:
partial_exit_market()

Positions before: {'C1_GAS_INC': 20, 'C1_OIL_CORP': 20, 'C1_FOSSIL_FUEL_ETF': -110, 'C2_GREEN_ENERGY_ETF': -475, 'C2_SOLAR_CO': 195, 'C2_WIND_LTD': 235}

PnL before: 1837.64

Trading out of positions
-- Inserting sell order for 20 lots of C1_GAS_INC, with limit price 0.10
-- Inserting sell order for 20 lots of C1_OIL_CORP, with limit price 93.80
-- Inserting buy order for 110 lots of C1_FOSSIL_FUEL_ETF, with limit price 166.10
-- Inserting buy order for 475 lots of C2_GREEN_ENERGY_ETF, with limit price 103.30
-- Inserting sell order for 195 lots of C2_SOLAR_CO, with limit price 237.60
-- Inserting sell order for 235 lots of C2_WIND_LTD, with limit price 113.30

Positions after: {'C1_GAS_INC': 0, 'C1_OIL_CORP': 0, 'C1_FOSSIL_FUEL_ETF': 0, 'C2_GREEN_ENERGY_ETF': 0, 'C2_SOLAR_CO': 0, 'C2_WIND_LTD': 0}

PnL after: 1668.64


In [ ]:
for i in exchange.get_positions(): # deleting the limit orders 
    exchange.delete_orders(i)
    

In [4]:
for i in exchange.get_positions():
    print(exchange.get_outstanding_orders(i))
    

{}
{}
{}
{}
{}
{}
